In [1]:
import os
import subprocess
import sys
sys.path.append('..')
from tools.formating.formating import *
from tools.utils.utils import *
from tools.qc.scanpy_qc import *
import numpy as np
import scipy

scanpy==1.9.5 anndata==0.9.2 umap==0.5.4 numpy==1.22.4 scipy==1.9.1 pandas==1.3.5 scikit-learn==1.3.1 statsmodels==0.14.0 igraph==0.10.8 pynndescent==0.5.10


In [2]:
# input = "/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_germ line.h5ad"
# input = "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad"
input = "/ps/ai-ready/data/error/hca_heart_neuronal_raw.h5ad"
adata = load_anndata(input)
adata

/ps/ai-ready/data/error/hca_heart_neuronal_raw.h5ad


AnnData object with n_obs × n_vars = 3961 × 33538
    obs: 'NRP', 'age_group', 'cell_source', 'cell_type', 'donor', 'gender', 'n_counts', 'n_genes', 'percent_mito', 'percent_ribo', 'region', 'sample', 'scrublet_score', 'source', 'type', 'version', 'cell_states', 'Used'
    var: 'gene_ids-Harvard-Nuclei', 'feature_types-Harvard-Nuclei', 'gene_ids-Sanger-Nuclei', 'feature_types-Sanger-Nuclei', 'gene_ids-Sanger-Cells', 'feature_types-Sanger-Cells', 'gene_ids-Sanger-CD45', 'feature_types-Sanger-CD45'
    uns: 'cell_type_colors'
    obsm: 'X_pca', 'X_umap'

In [ ]:
type(adata.X)

In [ ]:
isinstance(adata.X, csr_matrix)

In [ ]:
np.set_printoptions(threshold=np.inf)
adata.X.toarray()[5,:]

In [ ]:
type(adata.X.toarray())

In [ ]:
isinstance(adata.X.toarray(), np.ndarray)

In [ ]:
np.max(adata.X)

In [ ]:
np.min(adata.X)

In [ ]:
from scipy.sparse import csr_matrix, find
find(adata.X)

In [ ]:
from scipy import stats
stats.describe(adata.X.toarray())

In [ ]:
np.median(adata.X.toarray())

In [ ]:
min_genes=200
min_cells=3
target_sum=1e4
n_neighbors=10
n_pcs=40

In [ ]:
adata.var_names_make_unique() 

# Filtering low quality reads
sc.pp.filter_cells(adata, min_genes=min_genes)
sc.pp.filter_genes(adata, min_cells=min_cells)
# mitochondrial genes
adata.var['mt']=adata.var_names.str.startswith('MT-')
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes.
adata.var["hb"] = adata.var_names.str.contains(("^HB[^(P)]"))
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True)

adata.obs["outlier"] = (
    is_outlier(adata, "log1p_total_counts", 5)
    | is_outlier(adata, "log1p_n_genes_by_counts", 5)
    | is_outlier(adata, "pct_counts_in_top_20_genes", 5)
)
adata.obs.outlier.value_counts()

adata.obs["mt_outlier"] = is_outlier(adata, "pct_counts_mt", 3) | (
    adata.obs["pct_counts_mt"] > 8
)
adata.obs.mt_outlier.value_counts()

print(f"Total number of cells: {adata.n_obs}")
adata = adata[(~adata.obs.outlier) & (~adata.obs.mt_outlier)].copy()

print(f"Number of cells after filtering of low quality cells: {adata.n_obs}")
adata.X

In [ ]:
# adata=adata[adata.obs.n_genes_by_counts < 2500, :]
# adata=adata[adata.obs.pct_counts_mt < 5, :]

scrub = scr.Scrublet(adata.X, expected_doublet_rate = 0.076)
adata.obs['doublet_scores'], adata.obs['predicted_doublets'] = scrub.scrub_doublets(min_counts=2, min_cells=3, 
                                                        min_gene_variability_pctl=85, n_prin_comps=30)
adata.obs['predicted_doublets'].value_counts()
# adata=adata[adata.obs.predicted_doublets=="False", :]

sc.pp.normalize_total(adata, target_sum=target_sum)

sc.pp.log1p(adata)

In [ ]:
adata.X

In [ ]:
sc.pp.highly_variable_genes(adata)

In [ ]:
import scrublet as scr
scrub = scr.Scrublet(adata.X, expected_doublet_rate = 0.076)
adata.obs['doublet_scores'], adata.obs['predicted_doublets'] = scrub.scrub_doublets(min_counts=2, min_cells=3, 
                                                        min_gene_variability_pctl=85, n_prin_comps=30)
adata.obs['predicted_doublets'].value_counts()

In [ ]:
adata=adata[adata.obs.predicted_doublets==False, :]
adata

In [3]:
adata = run_scanpy_qc(adata)
adata

filtered out 11455 genes that are detected in less than 3 cells
Total number of cells: 3961
Number of cells after filtering of low quality cells: 3295
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.60
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 0.2%
Overall doublet rate:
	Expected   = 7.6%
	Estimated  = 0.0%
Elapsed time: 3.8 seconds
normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes
    finished (0:00:01)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
regressing out ['total_counts', 'pct_counts_mt']
    sparse input is densified and may lead to high memory use
    finished (0:00:50)
computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:00:01)
computing n

AnnData object with n_obs × n_vars = 3295 × 4909
    obs: 'NRP', 'age_group', 'cell_source', 'cell_type', 'donor', 'gender', 'n_counts', 'n_genes', 'percent_mito', 'percent_ribo', 'region', 'sample', 'scrublet_score', 'source', 'type', 'version', 'cell_states', 'Used', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'doublet_scores', 'predicted_doublets', 'leiden', 'cluster2'
    var: 'gene_ids-Harvard-Nuclei', 'feature_types-Harvard-Nuclei', 'gene_ids-Sanger-Nuclei', 'feature_types-Sanger-Nuclei', 'gene_ids-Sanger-Cells', 'feature_types-Sanger-Cells', 'gene_ids-Sanger-CD45', 'feature_types-Sanger-CD45', 'n_cells', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_co

In [ ]:
adata.obs

In [ ]:
train, test = sc_train_test_split(adata, train_fraction=0.8)

In [ ]:
train, val, test = sc_train_val_test_split(adata, train_fraction=0.8, validation_fraction=0.1, test_fraction=0.1)

In [ ]:
train

In [ ]:
val

In [ ]:
test

In [ ]:
adata.var

In [ ]:
list(adata.layers.keys())

In [ ]:
adata.n_vars

In [ ]:
adata.n_obs

In [ ]:
adata.obs_names.to_list()

In [ ]:
adata.var_names.to_list()

In [ ]:
list(adata.obsm.keys())

In [ ]:
adata.X

In [ ]:
type(adata.obs_keys())

In [ ]:
adata.var_keys()

In [ ]:
random_state=0
fraction=0.8
np.random.seed(random_state)
old_n_obs = adata.n_obs
old_n_obs
new_n_obs = int(fraction * old_n_obs)
obs_indices = np.random.choice(old_n_obs, size=new_n_obs, replace=False)
obs_indices

In [ ]:
test=adata[obs_indices].copy()
test

In [ ]:
test.var

In [ ]:
original_indices=np.arange(old_n_obs)
original_indices

In [ ]:
diff=np.setdiff1d(original_indices, obs_indices)
diff

In [ ]:
len(diff)

In [ ]:
np.random.shuffle(diff)
diff

In [ ]:
train=adata[diff].copy()
train

In [ ]:
train.obs